In [5]:
import modulos as md
import numpy as np
import constantes as cons
import os
import sys
lista_filtrada = []

#lista=md.lista_de_monedas()
#lista=md.filtradodemonedas ()
#lista=['CFXUSDT', 'BLZUSDT', 'APEUSDT', 'LTCUSDT', 'LINKUSDT', 'WLDUSDT', 'BNBUSDT', 'FLMUSDT', 'SEIUSDT', '1000PEPEUSDT', 'CYBERUSDT', 'SUIUSDT', 'ETHUSDT', 'RUNEUSDT', 'BCHUSDT', 'MATICUSDT', 'TRBUSDT', 'ENJUSDT', 'UNFIUSDT', 'OPUSDT', 'DOGEUSDT', 'PERPUSDT', 'TOMOUSDT', 'BTCUSDT', 'SNXUSDT', 'XRPUSDT', 'SOLUSDT']
lista=['JOEUSDT']

if len(lista)>1:
    imprimo=False
    debug=False
else:
    imprimo= True
    debug=True
for symbol in lista:    
    try:
        #data,porcentajeentrada = md.estrategia_santa(symbol,tp_flag = True)
        #data,porcentajeentrada = md.estrategia_trampa(symbol,tp_flag = True)
        data,porcentajeentrada = md.estrategia_atrapes(symbol, debug=debug)
        #data,porcentajeentrada = md.estrategia_oro('XAU',tp_flag = True)
        #######################################################################
        resultado = md.backtesting(data, plot_flag = imprimo,porcentajeentrada=porcentajeentrada)
        #resultado = md.backtestingsanta(data, plot_flag = imprimo, debug = True)
        ########################################################################
        #if resultado['Return [%]'] < 0: # para santa3 mazmorra
        #if data.disparo.iloc[-1]!=0: # imprime las monedas que estan en posible haz martillo
        #        print(symbol)
        if resultado['Return [%]'] > 0:
            #if ((resultado['Profit Factor'] > 2 or np.isnan(resultado['Profit Factor'])) and (resultado['Return [%]']/resultado['# Trades']) >=0.33):            
                lista_filtrada.append(symbol)
        print(f"{symbol} - Return [%]: {md.truncate(resultado['Return [%]'],2)}% - # Trades: {resultado['# Trades']} - Profit Factor: {resultado['Profit Factor']} - Win Rate [%]: {resultado['Win Rate [%]']}")
    except Exception as falla:
        _, _, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print("\nError: "+str(falla)+" - line: "+str(exc_tb.tb_lineno)+" - file: "+str(fname)+"\n")
        pass
for symbol in lista_filtrada:
    md.printandlog(cons.nombrelog,symbol,pal=1)
#md.dibuja_patrones_triangulos (data,998)    



JOEUSDT - CHG 24h: 9.24% ↑ - lineas: 0.2692 y 0.2788 - martillo: -1
          Open Time  martillo  disparo  escape  signal  take_profit  stop_loss  previous_martillo
2023-09-04 09:10:00         0   0.0000  0.0000       0     0.000000     0.0000                  0
2023-09-04 09:15:00         0   0.0000  0.0000       0     0.000000     0.0000                  0
2023-09-04 09:20:00         0   0.0000  0.0000       0     0.000000     0.0000                  0
2023-09-04 09:25:00         0   0.0000  0.0000       0     0.000000     0.0000                  0
2023-09-04 09:30:00         0   0.0000  0.0000       0     0.000000     0.0000                  0
2023-09-04 09:35:00         0   0.0000  0.0000       0     0.000000     0.0000                  0
2023-09-04 09:40:00         0   0.0000  0.0000       0     0.000000     0.0000                  0
2023-09-04 09:45:00         0   0.0000  0.0000       0     0.000000     0.0000                  0
2023-09-04 09:50:00         0   0.0000  0.0000   

C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


JOEUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan
